In [1]:
import os
import json
from dlfm_code.tester import test
import shutil
%pdb 


Automatic pdb calling has been turned ON


In [2]:
exp_names = ['tonic', 'mode', 'joint'];

training = ['single', 'multi'];
distribution = ['pd', 'pcd'];

bin_size = [7.5, 15.0, 25.0, 50.0, 100.0];
bin_size_str = ['7_5', '15_0', '25_0','50_0', '100_0'];

kernel_width = [0, 7.5, 15.0, 25.0, 50.0, 100.0]
kernel_width_str = ['0', '7_5', '15_0', '25_0','50_0', '100_0'];

dists = ['l1', 'l2', 'l3', 'bhat', 'dis_intersect', 'dis_corr'];

k_neighbors = [1, 3, 5, 10, 15];
k_neighbors_str = ['1', '3', '5', '10', '15'];

peak_ratio = [0.15];
peak_ratio_str = ['0_15'];

test_dir = './data/testing/';
folds = json.load(open('./data/folds.json', 'r'))

In [3]:
it = 0
total = 0
res_err = []
for ee in exp_names:
    for tt in training:
        for dd in distribution:
            for (bb, bbs) in zip(bin_size, bin_size_str):
                for (kk, kks) in zip(kernel_width, kernel_width_str):
                    for di in dists:
                        for (kn, kns) in zip(k_neighbors, k_neighbors_str):
                            param_str_list = [tt, dd, bbs, kks, di, kns, peak_ratio_str[0]]
                            param_str = '--'.join(param_str_list)
                            
                            for ff in range(10):
                                fold_dir = os.path.join(test_dir, ee, param_str, 'fold' + str(ff))
                                
                                if not os.path.isdir(fold_dir):
                                    if tt == 'single' and kn != 1:
                                        pass
                                    elif bb > kk * 3 and kk != 0 :
                                        pass
                                    else:
                                        print "Missing " + fold_dir
                                        import pdb
                                        pdb.set_trace()
                                else:
                                    res_file = os.path.join(fold_dir, 'results.json')
                                    try:
                                        temp_res = json.load(open(res_file, 'r'))
                                    except IOError:
                                        temp_res = dict()
                                        
                                    test_mbids = set(fold['source'] for fold in folds[ff][1]['testing'])
                                    total += 1
                                    
                                    if (test_mbids - set(temp_res.keys())):
                                        
                                        it += 1
                                        
                                        param_list = [bb, kk, dd, tt, ff, ee, di, kn, peak_ratio[0], 5, os.path.join('.', 'data')]
                                        
                                        res_err.append((res_file, param_list))
                                        # test(*param_list, overwrite=True)
                                        
                                        print res_file

In [4]:
print total
print it
json.dump(res_err, open('res_err.json', 'w'))

51840
0
